In [1]:
import torch
from hw_asr.text_encoder.ctc_char_text_encoder import CTCCharTextEncoder

In [2]:
text_encoder = CTCCharTextEncoder()
log_probs_length = 270
probs = torch.full((log_probs_length, len(text_encoder.char2ind)), 1 / len(text_encoder.char2ind))
for beam_size in [1, 5, 10, 20, 50, 70, 100, 120, 150]:
    print(f'{beam_size = }')
    %timeit text_encoder.ctc_beam_search(torch.log(probs), log_probs_length, beam_size=beam_size)

beam_size = 1
9.44 ms ± 382 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
beam_size = 5
56.3 ms ± 1.57 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
beam_size = 10
114 ms ± 8.07 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
beam_size = 20
237 ms ± 31.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
beam_size = 50
479 ms ± 15.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
beam_size = 70
678 ms ± 70.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
beam_size = 100
938 ms ± 69.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
beam_size = 120
1.07 s ± 21.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
beam_size = 150
1.35 s ± 31.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
